In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

In [19]:
# Read the matrices from the text file
with open('matrices_training_data.txt', 'r') as file:
    matrix_lines = file.readlines()

matrices = []
for i in range(0, len(matrix_lines), 12):
    matrix = np.array([list(map(int, line.strip().split())) for line in matrix_lines[i:i+12]])
    matrices.append(matrix)

# Read the energies from the text file
with open('energies_training_data.txt', 'r') as file:
    energy_lines = file.readlines()

energies = np.array([int(line.strip()) for line in energy_lines])

In [20]:
# Split the data into training and validation sets
train_matrices, val_matrices, train_energies, val_energies = train_test_split(
    matrices, energies, test_size=0.1, random_state=42)


In [21]:
# Reshape the matrices for the CNN model
train_matrices = np.array(train_matrices).reshape(-1, 12, 24, 1)
val_matrices = np.array(val_matrices).reshape(-1, 12, 24, 1)


In [22]:
# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(12, 24, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))  # Adjust the pooling size to (1, 2)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [31]:
# Compile the model
learning_rate = 0.010  # You can change this value as needed
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the specified optimizer
model.compile(optimizer=optimizer, loss='mae')

In [32]:
# Train the model
epochs = 10
batch_size = 32
model.fit(train_matrices, train_energies, batch_size=batch_size, epochs=epochs, validation_data=(val_matrices, val_energies))

Epoch 1/10
 3599/28125 [==>...........................] - ETA: 10:23 - loss: 13.7326

In [29]:
# Evaluate the model
val_loss = model.evaluate(val_matrices, val_energies)
print("Validation Loss:", val_loss)

   1/3125 [..............................] - ETA: 1:33 - loss: 12.1786

3125/3125 [==============================] - 13s 4ms/step - loss: 12.7514
Validation Loss: 12.751436233520508


In [30]:
# Make predictions on validation data
val_predictions = model.predict(val_matrices)
val_predictions = np.squeeze(val_predictions)  # Remove unnecessary dimensions

# Calculate accuracy
threshold = 0  # Threshold for considering a prediction as positive or negative
val_predictions[val_predictions > threshold] = 1
val_predictions[val_predictions <= threshold] = -1

accuracy = np.mean(val_predictions == val_energies)
print("Validation Accuracy:", accuracy)

3125/3125 [==============================] - 13s 4ms/step
Validation Accuracy: 0.0


TESTING ALGORITHM

In [13]:
init_random = np.random.random((12,24))
lattice_p = np.zeros((12, 24))
lattice_p[init_random>=0.5] = 1
lattice_p[init_random<0.5] = -1
lattice_p1 = lattice_p

In [14]:
# Reshape and normalize the testing matrices
lattice_p1 = lattice_p1.reshape(-1, 12, 24, 1)
lattice_p1 = lattice_p1 / 2.0 - 0.5


In [15]:
# Make predictions on the testing data
predictions = model.predict(lattice_p1)

# Print the predictions
for i, prediction in enumerate(predictions):
    print("Matrix", i+1, "Energy:", prediction)

1/1 [==============================] - 0s 57ms/step
Matrix 1 Energy: [-306.9257]


In [16]:
def get_energy(lattice):
    # applies the nearest neighbours summation
    kern = generate_binary_structure(2, 1) 
    kern[1][1] = False
    arr = -lattice * convolve(lattice, kern, mode='constant', cval=0)
    return arr.sum()    

In [17]:
get_energy(lattice_p)

NameError: name 'generate_binary_structure' is not defined